In [1]:
import numpy as np
import pandas as pd

#set cd to data folder with QC'ed rides
% cd /Users/fineiskid/Desktop/Access_Analysis_Rproject_local/data/


/Users/fineiskid/Desktop/Access_Analysis_Rproject_local/data


In [2]:
data = pd.read_csv("single_clean_day.csv")

In [3]:
data["ServiceDate"] = pd.to_datetime(data["ServiceDate"], format ='%Y-%m-%d %H:%M:%S', unit = "D")

In [4]:
def add_TimeWindows(data, windowsz):
    '''calculate time windows (pickup and dropoff)
        from SchTime and ETA.
        data is subsetted schedule data from a day.
        windowsz is size of pickup/dropoff window in seconds'''

    etas = data["ETA"]
    schtime =data["SchTime"]
    schtime[schtime<0] = np.nan
    data["PickupStart"] = 0; data["PickupEnd"] = 0
    data["DropoffStart"] = 0; data["DropoffEnd"] = 0
    for x in range(0, len(etas)):

        #make dropoff window when there's no required drop off time
        if (data["Activity"].iloc[x] == 1) & (data["ReqLate"].iloc[x] <0):
            data["DropoffStart"].iloc[x] = data["ETA"].iloc[x]-3600
            data["DropoffEnd"].iloc[x] = data["ETA"].iloc[x]+3600
        
        #make dropoff window when there IS a required drop off time: 1hr before ReqLate time
        if (data["Activity"].iloc[x] == 1) & (data["ReqLate"].iloc[x] >0):
            data["DropoffStart"].iloc[x] = data["ETA"].iloc[x]-3600
            data["DropoffEnd"].iloc[x] = data["ReqLate"].iloc[x]  
        
        #schtime is in the middle of the pick up window
        if data["Activity"].iloc[x] == 0:
            data["PickupStart"].iloc[x] = schtime.iloc[x]-(windowsz/2)
            data["PickupEnd"].iloc[x] = schtime.iloc[x]+(windowsz/2)

    return data

In [74]:
# SAVE SINGLE DAY'S RIDES WITH TIME WINDOWS
data = add_TimeWindows(data, 60*30)
data.to_csv(path = "/Users/fineiskid/Desktop/single_day_TimeWindows.csv", index = False)

47.509999999999998

In [6]:
def get_URIDs(data, broken_Run, resched_init_time):
    '''get unscheduled request id's from broken bus,
        based on when we're allowed to first start rescheduling.
        resched_init_time is in seconds, marks the point in time we can begin considering reinserting new requests.
        broken_Run is number of run that breaks
        data is today's scheduling data

        RETURN: list of URIDs'''
    
    #all rides that exist past time we're allowed to begin rescheduling
    leftover = data[data["ETA"] >= resched_init_time]
    leftover = leftover[(leftover["Activity"] != 6) & (leftover["Activity"] != 16) & (leftover["Activity"] != 3)]
    
    #rides that were scheduled to be on broken bus past resched_init_time
    pickmeup = leftover[leftover["Run"]==broken_Run]
    clients = pickmeup["ClientId"].unique()
    clients = clients[~(np.isnan(clients))]
    rmClients = []
    
    #remove people who would were scheduled to be on bus before resched_init_time
    for cli in clients:
        onoff = pickmeup[pickmeup["ClientId"]==cli]
        if onoff.shape[0] == 1:
            rmClients.append(cli)
    unsched = pickmeup[~pickmeup["ClientId"].isin(rmClients)]

    print("There are %s rides left to be scheduled on broken run %s" % (unsched.shape[0], broken_Run))

    class URID:
        def __init__(self, BookingId, Run, PickUpCoords, DropOffCoords, PickupStart, PickupEnd, DropoffStart, DropoffEnd, SpaceOn, MobAids):
            self.BookingId= BookingId
            self.Run = Run
            self.PickUpCoords = PickUpCoords
            self.DropOffCoords = DropOffCoords
            self.PickupStart = PickupStart
            self.PickupEnd = PickupEnd
            self.DropoffStart = DropoffStart
            self.DropoffEnd = DropoffEnd
            self.SpaceOn = SpaceOn
            self.MobAids = MobAids


    diffIDs = unsched.BookingId.unique()
    saveme = []

    #save separate URID's in a list
    for ID in diffIDs:
        temp = URID(BookingId = ID,
            Run = broken_Run,
            PickUpCoords = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["LAT", "LON"]].iloc[0,],
            DropOffCoords = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["LAT", "LON"]].iloc[1,],
            PickupStart = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["PickupStart"]].iloc[0,],
            PickupEnd = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["PickupEnd"]].iloc[0,],
            DropoffStart = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["DropoffStart"]].iloc[0,],
            DropoffEnd = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["DropoffEnd"]].iloc[0,],
            SpaceOn = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["SpaceOn"]].iloc[0,],
            MobAids = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["MobAids"]].iloc[0,])
        saveme.append(temp)

    return saveme

In [7]:
broken_Run = data.Run.unique()[0]
resched_init_time = 14*60*60
urids = get_URIDs(data, broken_Run, resched_init_time)

There are 10 rides left to be scheduled on broken run 680SEB


In [8]:
def get_busRuns(data, Run, resched_init_time):
    ''' take pd.DataFrame from add_Time_Windows.py and create busRun object for specified Run number,
    for all stops occurring at or after resched_init_time.

    RETURN: busRun object for specified Run.'''

    #subset based on matching Run number, and subset for stops only at or after resched_init_time
    print("Getting remaing rides for run %s" % Run)
    dataSub = data[(data["Run"] == Run) & (data["ETA"] >= resched_init_time)]
    #subset only the rides that aren't 6, 16, or 3:
    leave = dataSub.index.min()
    gas = dataSub[(dataSub["Activity"]==6)|(dataSub["Activity"]==16)|(dataSub["Activity"]==3)].index.min()
    busRun = dataSub.loc[leave:(gas-1)]

    return busRun

In [9]:
#Test get_busRuns subsetting routine
testRun = data.Run.unique()[2]
bus181SEB = get_busRuns(data, testRun, resched_init_time)

Getting remaing rides for run 181SEB


In [10]:
#list urid attributes:
urids[0].__dict__.keys()

['PickupEnd',
 'Run',
 'BookingId',
 'MobAids',
 'DropoffStart',
 'DropoffEnd',
 'PickUpCoords',
 'SpaceOn',
 'DropOffCoords',
 'PickupStart']

In [13]:
def time_insertions(Run_Schedule, URID, pickUpDropOff = True):
    '''URID: of class URID, has bookingId, pickUpLocation, dropOffLocation, etc.
        Run_Schedule: Schedule (pd.Data.Frame) of the run on which we're trying to insert the URID
        RETURN: pd.Data.Frame corresponding to nodes with overlapping pickup/dropoff windows'''

    #CURRENT ISSUES: RUNSCHEDULE SHOULD HAVE PRECEDING NODES, AT LEAST ONE PRIOR TO RESCHED_INIT_TIME

    if pickUpDropOff:
        Start = URID.PickupStart[0]
        End = URID.PickupEnd[0]
    else:
        Start = URID.DropoffStart[0]
        End = URID.DropoffEnd[0]
        
    crossover = []
    
    for jj in range(Run_Schedule.shape[0]):
        #Checking if a Run's PickupWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 0:
            PUE = Run_Schedule.PickupEnd.iloc[jj]; PUS = Run_Schedule.PickupStart.iloc[jj]
            #simple, unequal overlap
            if (PUE > Start) & (PUS < End):
                crossover.append(Run_Schedule.index[jj])
            # equal or strictly within [PUS, PUE]
            if (PUE <= End) & (PUS >= Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered by [PUS, PUE] and then some on both sides
            if (PUE > End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on left side
            if (PUE == End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on right side
            if (PUS == Start) & (PUE > End):
                crossover.append(Run_Schedule.index[jj])
                
        #Checking if a Run's DropoffWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 1:
            DOE = Run_Schedule.DropoffEnd.iloc[jj]; DOS = Run_Schedule.DropoffStart.iloc[jj]
            if (DOE > Start) & (DOS < End):
                crossover.append(Run_Schedule.index[jj])
            if (DOE <= End) & (DOS >= Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE > End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE == End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOS == Start) & (DOE > End):
                crossover.append(Run_Schedule.index[jj])

    #Get rid of cases that repeat themselves:
    crossover = set(crossover)
    print("Need to service URID within %s sec to %s sec" % (Start, End))
    print("These indices of Run_Schedule overlap with URID: %s" % crossover)

    return Run_Schedule.loc[crossover]

pickUpDropOff = True; URID = urids[0]; Run_Schedule = bus181SEB

In [79]:
from haversine import haversine
inserts = time_insertions(Run_Schedule, URID, pickUpDropOff)
overlap_LAT = inserts.LAT.tolist()
overlap_LON = inserts.LON.tolist()
if pickUpDropOff:
    URID_loc = ([URID.PickUpCoords["LAT"], URID.PickUpCoords["LON"]])
else:
    URID_loc = ([URID.DropOffCoords["LAT"], URID.PickUpCoords["LON"]])

Need to service URID within 51300.0 sec to 53100.0 sec
These indices of Run_Schedule overlap: set([52, 53, 54, 55, 58, 59])


In [80]:
for k in range(len(overlap_LAT)):
    point = (overlap_LAT[k], overlap_LON[k])
    dist = haversine(point, URID_loc, miles=True)
    print(dist)

23.8160392766
22.3902177184
22.3902177184
20.4168699431
18.5852218387
19.827313131


In [85]:
data[data.Run != URID.Run].shape


(7322, 30)